In [1]:
import os
os.chdir("Y:\Python\Mega_IT\F.Machine_Learning/3.Unsupervised_Learning/4.Surprise")
os.getcwd()

'Y:\\Python\\Mega_IT\\F.Machine_Learning\\3.Unsupervised_Learning\\4.Surprise'

In [2]:
import sys
sys.path.append("Y:\Python\Mega_IT")

import warnings
warnings.filterwarnings('ignore')

from yjk_module.preprocessing import *
from yjk_module.ml_module import *

from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
from hossam.util import *
from hossam.plot import *
from hossam.analysis import *
from hossam.classification import *

In [4]:
from surprise import BaselineOnly, Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split as s_train_test_split, cross_validate as s_cross_validate, GridSearchCV as s_GridSearchCV, RandomizedSearchCV as s_RandomizedSearchCV

- 추천시스템 : 비지도 학습, 내부적으로는 SGD를 사용
    - 사용자에 대한 인구통계학적인 정보를 사용함 : 나이 / 성별 / 인종을 고려하여 집단(cluster)를 정의하고 이를 분류에 활용
    - Recommander system : 비슷한 인구통계학적 특성을 보이는 사람은 취향도 비슷하리라 판단함 (개개인별로 취향이 제각각이므로 정확도가 떨어짐)
    - Content filtering : 아이템의 내용에 집중하여 해당 기본 정보를 활용
    - Collaborative filtering : 타깃 유저의 데이터와 다른 유저의 데이터를 모두 사용
    - 이를 사용하기 위한 surprise 패키지는 사이킷런을 기반으로 하고 있음

In [5]:
origin1 = pd.read_excel("./data/movie_ratings.xlsx")

In [6]:
movies = pd.read_excel("./data/movie_ratings.xlsx", sheet_name = 'movies')
links = pd.read_excel("./data/movie_ratings.xlsx", sheet_name = 'links')
tags = pd.read_excel("./data/movie_ratings.xlsx", sheet_name = 'tags')

- surprise 패키지는 사용자번호, 아이템번호, 평점번호 3가지의 고정된 데이터를 요구함

In [7]:
origin1.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
df1 = origin1.drop('timestamp', axis = 1)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100836 non-null  int64  
 1   movieId  100836 non-null  int64  
 2   rating   100836 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 2.3 MB


In [9]:
df1.describe()

,userId,movieId,rating
count,100836.000000,100836.000000,100836.000000
mean,326.127564,19435.295718,3.501557
std,182.618491,35530.987199,1.042529
min,1.000000,1.000000,0.500000
25%,177.000000,1199.000000,3.000000
50%,325.000000,2991.000000,3.500000
75%,477.000000,8122.000000,4.000000
max,610.000000,193609.000000,5.000000


- 평점 분포를 알려주는 reader 객체 사용

In [10]:
reader = Reader(rating_scale = (0.5, 5.0))

- 데이터프레임을 surprise 에서 사용가능한 형태로 가공하는 Dataset 객체에 Reader 객체와 함께 변환

In [11]:
data = Dataset.load_from_df(df1, reader)

- train / test 분리 : Surprise 패키지 자체 함수 사용

In [12]:
train, test = s_train_test_split(data, test_size = 0.2, random_state = 0)

- 추천 기본 모형 구현

In [13]:
estimator = BaselineOnly()
estimator.fit(train)

Estimating biases using als...


- 예측 및 성능 평가

In [14]:
from surprise.accuracy import rmse, mae

In [15]:
pred = estimator.test(test)

In [16]:
pred[:5]

[Prediction(uid=548, iid=1196, r_ui=3.5, est=4.4688569029292955, details={'was_impossible': False}),
 Prediction(uid=590, iid=1252, r_ui=3.0, est=3.8284627528627215, details={'was_impossible': False}),
 Prediction(uid=8, iid=32, r_ui=3.0, est=4.003777018267144, details={'was_impossible': False}),
 Prediction(uid=217, iid=2993, r_ui=3.0, est=2.9884116522794253, details={'was_impossible': False}),
 Prediction(uid=51, iid=2613, r_ui=5.0, est=3.4693253210141344, details={'was_impossible': False})]

- uid : 사용자 번호
- iid : 아이템 번호
- r_ui : 해당 사용자가 실제로 부여한 평점
- est : 예측 평점

In [17]:
score = rmse(pred)

RMSE: 0.8685


In [18]:
upred = estimator.predict(uid = 603, iid = 3996)
upred

Prediction(uid=603, iid=3996, r_ui=None, est=3.875007655666056, details={'was_impossible': False})

In [19]:
upred.est

3.875007655666056

In [20]:
rmse(pred), mae(pred)

RMSE: 0.8685
MAE:  0.6693


(0.8685101467336913, 0.6693169488048113)

- Surprise 객체는 파라미터 설정을 bsl_options에 딕셔너리를 입력하여 진행

In [21]:
estimator = BaselineOnly(bsl_options = {
    "method" : "als",
    "n_epochs": 10,
    "reg_u" : 10,
    "reg_i" : 15
})
cv_result = s_cross_validate(estimator, data, measures = ['RMSE', 'MAE'], cv = 5, verbose = True) # verbose 파라미터는 결과를 출력할지 여부 결정

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8778  0.8759  0.8796  0.8759  0.8744  0.8767  0.0018  
MAE (testset)     0.6760  0.6743  0.6771  0.6790  0.6766  0.6766  0.0015  
Fit time          0.08    0.07    0.06    0.07    0.09    0.07    0.01    
Test time         0.14    0.05    0.05    0.05    0.05    0.07    0.04    


In [22]:
cv_result

{'test_rmse': array([0.87777946, 0.8758704 , 0.87960586, 0.87590708, 0.87441509]),
 'test_mae': array([0.67598986, 0.67433018, 0.67714718, 0.67904282, 0.67660337]),
 'fit_time': (0.07506847381591797,
  0.07006335258483887,
  0.06405830383300781,
  0.06906294822692871,
  0.08507776260375977),
 'test_time': (0.14263606071472168,
  0.04604172706604004,
  0.04604172706604004,
  0.04904484748840332,
  0.04604291915893555)}

In [23]:
print("RMSE(mean) :", cv_result['test_rmse'].mean())
print("MAE(mean) : ", cv_result['test_mae'].mean())

RMSE(mean) : 0.8767155795156217
MAE(mean) :  0.6766226836114304


- GridSearch(RandomizedSearch)

In [24]:
params = {
    'bsl_options': {
        "method" : ['als', 'sgd'], # 알고리즘
        "n_epochs" : [10, 20], # 에포크
        "reg_u" : [10, 12], # 사용자에 대한 정규화 매개변수
        "reg_i" : [15, 20] # 아이템에 대한 정규화 매개변수
    }
}

grid = s_GridSearchCV(BaselineOnly,
                           param_grid = params,
                           measures = ['RMSE', 'MAE'],
                           cv = 5, n_jobs = -1)

grid.fit(data)

- 지표별로 가장 우수한 모델과 파라미터, 점수 제공

In [25]:
grid.best_estimator

{'rmse': <surprise.prediction_algorithms.baseline_only.BaselineOnly at 0x15411f29c60>,
 'mae': <surprise.prediction_algorithms.baseline_only.BaselineOnly at 0x15411f29a20>}

In [26]:
grid.best_score

{'rmse': 0.8700752153107182, 'mae': 0.6687645434441856}

In [27]:
grid.best_params

{'rmse': {'bsl_options': {'method': 'sgd',
   'n_epochs': 20,
   'reg_u': 10,
   'reg_i': 15}},
 'mae': {'bsl_options': {'method': 'sgd',
   'n_epochs': 20,
   'reg_u': 10,
   'reg_i': 15}}}

In [28]:
best_est = grid.best_estimator['rmse']

- 이외 다수 알고리즘 존재 (KNNBasic, SVD 등)
    - 사용법은 베이스라인과 동일, 하이퍼파라미터만 레퍼런스를 확인하여 별개로 입력

In [29]:
params = {
    "bls_options" : {
        "k" : [30, 40, 50],
        "min_k" : [1, 2, 3],
        "method" : ["als", 'sgd'],
        "n_epochs" : [20, 40]
    },
    "sim_options" : {
        "name" : ["msd", "pearson", "pearson_baseline"]
    }
    
}

grid = s_RandomizedSearchCV(KNNBasic,
                           param_distributions = params,
                           measures = ['RMSE', 'MAE'],
                           cv = 5, n_jobs = -1)

grid.fit(data)

In [30]:
grid.best_params

{'rmse': {'bls_options': 'method',
  'sim_options': {'name': 'msd', 'user_based': True}},
 'mae': {'bls_options': 'method',
  'sim_options': {'name': 'msd', 'user_based': True}}}

In [31]:
grid.best_estimator

{'rmse': <surprise.prediction_algorithms.knns.KNNBasic at 0x15411c82fb0>,
 'mae': <surprise.prediction_algorithms.knns.KNNBasic at 0x15408c22440>}

In [32]:
estimator = grid.best_estimator['rmse']
estimator.fit(train)
pred = estimator.test(test)
pred[:5]

Computing the msd similarity matrix...
Done computing similarity matrix.


[Prediction(uid=548, iid=1196, r_ui=3.5, est=4.119660326001586, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=590, iid=1252, r_ui=3.0, est=4.339040365451278, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=8, iid=32, r_ui=3.0, est=4.045567648098331, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=217, iid=2993, r_ui=3.0, est=3.594863398424905, details={'actual_k': 15, 'was_impossible': False}),
 Prediction(uid=51, iid=2613, r_ui=5.0, est=2.597772394867796, details={'actual_k': 7, 'was_impossible': False})]

In [33]:
estimator.predict(uid = 603, iid = 3996)

Prediction(uid=603, iid=3996, r_ui=None, est=4.087228116331484, details={'actual_k': 40, 'was_impossible': False})

- SVD (행렬분해 기반 잠재요인 협업 필터링)
    - 개선된 버전 : SVDpp

In [34]:
from surprise import SVD, SVDpp

In [35]:
params = {
    "n_epochs" : [20, 40, 50],
    "n_factors" : [100, 200, 300] # 요인 수
}

grid = s_GridSearchCV(SVD,
                           param_grid = params,
                           measures = ['RMSE', 'MAE'],
                           cv = 5, n_jobs = -1)

grid.fit(data)

In [36]:
grid.cv_results['params']

[{'n_epochs': 20, 'n_factors': 100},
 {'n_epochs': 20, 'n_factors': 200},
 {'n_epochs': 20, 'n_factors': 300},
 {'n_epochs': 40, 'n_factors': 100},
 {'n_epochs': 40, 'n_factors': 200},
 {'n_epochs': 40, 'n_factors': 300},
 {'n_epochs': 50, 'n_factors': 100},
 {'n_epochs': 50, 'n_factors': 200},
 {'n_epochs': 50, 'n_factors': 300}]

In [37]:
pd.DataFrame(grid.cv_results).head()

,split0_test_rmse,split1_test_rmse,split2_test_rmse,split3_test_rmse,split4_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,...,mean_test_mae,std_test_mae,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_n_epochs,param_n_factors
0,0.881357,0.871515,0.876339,0.873076,0.870639,0.874585,0.003904,1,0.677376,0.672051,...,0.672173,0.003851,1,0.568023,0.033848,0.121310,0.004669,"{'n_epochs': 20, 'n_factors': 100}",20,100
1,0.885728,0.875037,0.880173,0.882694,0.874257,0.879578,0.004401,5,0.681299,0.674365,...,0.676112,0.004387,6,1.364814,0.081925,0.175461,0.041455,"{'n_epochs': 20, 'n_factors': 200}",20,200
2,0.893582,0.880227,0.889486,0.880869,0.879791,0.884791,0.005666,9,0.688865,0.677762,...,0.681201,0.005442,9,2.171156,0.092340,0.223806,0.037220,"{'n_epochs': 20, 'n_factors': 300}",20,300
3,0.884210,0.879142,0.882687,0.881844,0.874413,0.880459,0.003442,6,0.678983,0.674280,...,0.675694,0.003509,5,1.854194,0.126378,0.200383,0.018836,"{'n_epochs': 40, 'n_factors': 100}",40,100
4,0.887433,0.873643,0.879723,0.879405,0.873064,0.878653,0.005199,3,0.681568,0.672918,...,0.675308,0.004123,4,3.130769,0.099068,0.155843,0.031294,"{'n_epochs': 40, 'n_factors': 200}",40,200


In [38]:
estimator = grid.best_estimator['rmse']
estimator.fit(train)
pred = estimator.test(test)
pred[:5]

[Prediction(uid=548, iid=1196, r_ui=3.5, est=4.399761500431599, details={'was_impossible': False}),
 Prediction(uid=590, iid=1252, r_ui=3.0, est=3.996613711512641, details={'was_impossible': False}),
 Prediction(uid=8, iid=32, r_ui=3.0, est=3.8912654751169495, details={'was_impossible': False}),
 Prediction(uid=217, iid=2993, r_ui=3.0, est=3.062041650957773, details={'was_impossible': False}),
 Prediction(uid=51, iid=2613, r_ui=5.0, est=3.6097223759482078, details={'was_impossible': False})]

In [43]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


- 데이터 재학습

In [44]:
train = data.build_full_trainset()
estimator.fit(train)

- 9번 사용자가 본 영화

In [52]:
seen_movies = origin1[origin1['userId'] == 9] ['movieId'].tolist()
print(seen_movies)
print(f"9번 사용자가 본 영화 수 : {len(seen_movies)}")

[41, 187, 223, 371, 627, 922, 923, 1037, 1095, 1198, 1270, 1674, 1987, 2011, 2012, 2023, 2300, 2877, 2901, 3173, 3328, 3735, 4131, 4558, 4993, 5218, 5378, 5445, 5447, 5451, 5481, 5507, 5841, 5843, 5872, 5890, 5891, 5893, 5902, 5952, 5956, 5962, 5965, 5988, 6001, 6044]
9번 사용자가 본 영화 수 : 46


- 9번 사용자가 보지 않은 영화들 (추천 대상)

In [47]:
unseen_movies = movies[movies['movieId'].isin(seen_movies)]
unseen_movies.head()

,movieId,title,genres
37,41,Richard III (1995),Drama|War
158,187,Party Girl (1995),Comedy
190,223,Clerks (1994),Comedy
329,371,"Paper, The (1994)",Comedy|Drama
532,627,"Last Supper, The (1995)",Drama|Thriller


- 예상평점

In [60]:
unseen_movies['ratings'] = unseen_movies['movieId'].apply(lambda x : estimator.predict(uid = 9, iid = x).est)
unseen_movies.head()

,movieId,title,genres,ratings
37,41,Richard III (1995),Drama|War,3.337392
158,187,Party Girl (1995),Comedy,3.336210
190,223,Clerks (1994),Comedy,3.810576
329,371,"Paper, The (1994)",Comedy|Drama,2.986796
532,627,"Last Supper, The (1995)",Drama|Thriller,3.276176


In [61]:
unseen_movies.sort_values('ratings', ascending=False).head(10)

,movieId,title,genres,ratings
900,1198,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure,4.368629
4117,5902,Adaptation (2002),Comedy|Drama|Romance,4.181557
3638,4993,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy,4.156005
4137,5952,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy,4.146906
1711,2300,"Producers, The (1968)",Comedy,4.137724
705,923,Citizen Kane (1941),Drama|Mystery,4.116583
704,922,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),Drama|Film-Noir|Romance,4.053415
969,1270,Back to the Future (1985),Adventure|Comedy|Sci-Fi,4.007553
190,223,Clerks (1994),Comedy,3.810576
2494,3328,Ghost Dog: The Way of the Samurai (1999),Crime|Drama,3.787963


- 신규 회원은 예측 불가